# Chat With Anything - From PDFs Files to Image Documents

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
import sys
print(sys.path)
path = "\\anaconda3\\Lib\\site-packages\\"
sys.path.append(path)

['D:\\workspace\\NLP_tasks', 'C:\\Users\\bhise\\new_project\\library', 'D:\\workspace\\NLP_tasks', 'C:\\Users\\bhise\\anaconda3\\python310.zip', 'C:\\Users\\bhise\\anaconda3\\DLLs', 'C:\\Users\\bhise\\anaconda3\\lib', 'C:\\Users\\bhise\\anaconda3', '', 'C:\\Users\\bhise\\AppData\\Roaming\\Python\\Python310\\site-packages', 'C:\\Users\\bhise\\anaconda3\\lib\\site-packages', 'C:\\Users\\bhise\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\bhise\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\bhise\\anaconda3\\lib\\site-packages\\Pythonwin']


In [2]:
%%bash 

pip -q install langchain==0.0.251
pip -q install openai tiktoken
pip -q install pytesseract pypdf

### Chat & Query your PDF files
- Detect Document Type

In [4]:

!pip install filetype
from filetype import guess

def detect_document_type(document_path):
    
    guess_file = guess(document_path)
    file_type = ""
    image_types = ['jpg', 'jpeg', 'png', 'gif']
    
    if(guess_file.extension.lower() == "pdf"):
        file_type = "pdf"
        
    elif(guess_file.extension.lower() in image_types):
        file_type = "image"
        
    else:
        file_type = "unkown"
        
    return file_type

In [6]:
research_paper_path = ".\\workspace\\NLP_tasks\\NLP_tasks_with_Python\\data\\bert_research-paper.pdf"
article_information_path = "..\\workspace\\NLP_tasks\\NLP_tasks_with_Python\\data\\saruman.png"

print(f"Research Paper Type: {detect_document_type(research_paper_path)}")
print(f"Article Information Document Type: {detect_document_type(article_information_path)}")

Research Paper Type: pdf
Article Information Document Type: image


### Extract Documents Content

In [8]:
from langchain.document_loaders.image import UnstructuredImageLoader
from langchain.document_loaders import UnstructuredFileLoader


def extract_file_content(file_path):
    
    file_type = detect_document_type(file_path)
    
    if(file_type == "pdf"):
        loader = UnstructuredFileLoader(file_path)
        
    elif(file_type == "image"):
        loader = UnstructuredImageLoader(file_path)
        
    documents = loader.load()
    documents_content = '\n'.join(doc.page_content for doc in documents)
    
    return documents_content

In [34]:
path = "C:\\Users\\bhise\\anaconda3\\Lib\\site-packages\\pytesseract\\"

path1 = "C:\\Program Files\\Tesseract-OCR\\" 
    
sys.path.append(path)
sys.path.append(path1)
pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

In [35]:
research_paper_content = extract_file_content(research_paper_path)
article_information_content = extract_file_content(article_information_path)

In [36]:
research_paper_content

'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding\n\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova Google AI Language {jacobdevlin,mingweichang,kentonl,kristout}@google.com\n\nAbstract\n\nThere are two existing strategies for apply- ing pre-trained language representations to down- stream tasks: feature-based and ﬁne-tuning. The feature-based approach, such as ELMo (Peters et al., 2018a), uses task-speciﬁc architectures that include the pre-trained representations as addi- tional features. The ﬁne-tuning approach, such as the Generative Pre-trained Transformer (OpenAI GPT) (Radford et al., 2018), introduces minimal task-speciﬁc parameters, and is trained on the downstream tasks by simply ﬁne-tuning all pre- trained parameters. The two approaches share the same objective function during pre-training, where they use unidirectional language models to learn general language representations.\n\nWe introduce a new language representa- tion model ca

In [40]:
nb_characters = 400

print(f"First {nb_characters} Characters of the Paper: \n{research_paper_content[:nb_characters]}...")
print("---"*5)
print(f"First {nb_characters} Characters of Article Information Document :\n {article_information_content[:nb_characters]}...\")\n")

First 400 Characters of the Paper: 
BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding

Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova Google AI Language {jacobdevlin,mingweichang,kentonl,kristout}@google.com

Abstract

There are two existing strategies for apply- ing pre-trained language representations to down- stream tasks: feature-based and ﬁne-tuning. The feature-based approach, such as ...
---------------
First 400 Characters of Article Information Document :
 9 CONVERSATIONAL TRICKS TO MAKE PEOPLE THINK YOU ARE COOL AND SARUMAN

INSTEAD OF SAYING THIS}



»{sA...")



### Chat Implementation
- Create Chunks

In [41]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

In [42]:
research_paper_chunks = text_splitter.split_text(research_paper_content)
article_information_chunks = text_splitter.split_text(article_information_content)

print(f"# Chunks in Research Paper: {len(research_paper_chunks)}")
print(f"# Chunks in Article Document: {len(article_information_chunks)}")

Created a chunk of size 1039, which is longer than the specified 1000
Created a chunk of size 1064, which is longer than the specified 1000


# Chunks in Research Paper: 83
# Chunks in Article Document: 1


### Create Embeddings

In [46]:
from langchain.embeddings.openai import OpenAIEmbeddings
import os

os.environ["OPENAI_API_KEY"] = "<YOUR KEY>"

embeddings = OpenAIEmbeddings()

### Create Vector Index

In [47]:
from langchain.vectorstores import FAISS

def get_doc_search(text_splitter):
    
    return FAISS.from_texts(text_splitter, embeddings)

In [48]:
doc_search_paper = get_doc_search(research_paper_chunks)
print(doc_search_paper)

### Start chatting with your document

In [49]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(OpenAI(), chain_type = "map_rerank",  
                      return_intermediate_steps=True)

def chat_with_file(file_path, query):
    
    file_content = extract_file_content(file_path)
    file_splitter = text_splitter.split_text(file_content)
    
    document_search = get_doc_search(file_splitter)
    documents = document_search.similarity_search(query)
    
    results = chain({
                        "input_documents":documents, 
                        "question": query
                    }, 
                    return_only_outputs=True)
    results = results['intermediate_steps'][0]
    
    return results

### Chat with your image file

In [50]:
query = "What is the document about"

results = chat_with_file(article_information_path, query)

answer = results["answer"]
confidence_score = results["score"]

print(f"Answer: {answer}\n\nConfidence Score: {confidence_score}")

Answer:  The document is about 9 conversational tricks to make people think you are cool and Saruman.

Confidence Score: 100


### Chat with your PDF File

In [52]:
query = "Why is the paper about?"

results = chat_with_file(research_paper_path, query)

answer = results["answer"]
confidence_score = results["score"]

print(f"Answer: {answer}\n\nConfidence Score: {confidence_score}")

Created a chunk of size 1039, which is longer than the specified 1000
Created a chunk of size 1064, which is longer than the specified 1000


Answer:  The paper is about BERT, a conceptually simple yet empirically powerful model which obtains new state-of-the-art results on eleven natural language processing tasks.

Confidence Score: 90
